In [12]:
import pandas as pd
import urllib.request
import warnings

changes made 5/30: (feel free to look at github history if you want to revert some stuff)
- loaded in court_list and processed judges outside of function
- loaded in detail_codes for eventual mapping 
- used map instead of merge to replace court id with court name outside of function - looks cleaner
- took out all the stuff with the judges dataframe since it's already been moved over to the script
- creating a function for mapping codes with actual meanings
    - started the probation type column b/c "No Probation" and "NP" was my mistake when scraping

In [7]:
#read in courts and judges
court_list = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/courts.csv")
judges = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/judges.csv")
# codes = pd.read_csv("https://github.com/christinegu27/State-Sentencing-Project/blob/main/CSV%20Processing/edited_codes.csv",error_bad_lines=False)
#dictionary for replacing the court id with the court name
court_code = dict(court_list[["Court ID", "Court Name"]].values)
codes=pd.read_csv("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")

In [8]:
#Replace codes in the dataframe with their actual meaning
#Ex. replace "W" with "White" for defendant race
def map_values(df):
    d = dict()
    f = open("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")

    for line in f:
        line = line.strip('\n')
        line = line.split(",")
        df=df.replace({line[2]:{line[0]:line[1]}})
    return df

In [9]:
def case_matcher(cases):
    """
    cases: a dataframe of cases for 1 specifc court
    """
#     case_url="/Users/hinaljajal/Downloads/{}.csv".format(court+"_cases")
#     cases=pd.read_csv(case_url)
    
    cases["Year"]=cases["Last Hearing Date"].str[-4:]

    #checks if there are are different judges with the same initials
    judges1 = judges.drop_duplicates(subset=['Judge Full Name'])
    boolean = judges1.duplicated(subset=['Judge']).any()
    
    #Merges the columns by the judge and the hearing year
    cases['Year']=cases['Year'].astype(int)
    cases['Judge']=cases['Judge'].astype(str)
    judges_court['Year']=judges_court['Year'].astype(int)
    judges_court['Judge']=judges_court['Judge'].astype(str)


    cases = cases.merge(judges_court, how="left",on = ["Judge", "Year"])
        
    #Data cleaning
    cases['Race'] = cases['Race'].fillna("U") #U is the code for unknowns 
    cases['Sentence Y'] = cases['Sentence Y'].fillna(0)
    cases['Sentence M'] = cases['Sentence M'].fillna(0)
    cases['Sentence D'] = cases['Sentence D'].fillna(0)
    cases['Probation D'] = cases['Probation D'].fillna(0)
    cases['Probation Y'] = cases['Probation Y'].fillna(0)
    cases['Probation M'] = cases['Probation M'].fillna(0)
    cases['Judge Full Name'] = cases['Judge Full Name'].fillna("Unknown") #Nans are filled with unknowns 

    #dropping the name column
    cases=cases.drop(['Name'], axis=1)
    
    #Saves the dataframe to a csv
    file_name = '/Users/hinaljajal/Downloads/{}.csv'.format(court)
    cases.to_csv(file_name, index = False)
    
    return cases

In [10]:
separate_court_data = []
for court in court_list["Court ID"]:
    try:
        data = pd.read_csv("/Users/hinaljajal/Downloads/finished_courts/{}.csv".format(court))
    except FileNotFoundError:
        continue
    data = map_values(data)
    #Slices the judges for the particular court 
    judges_court = judges[judges["Court Name"]==court_code[court]]
    #replace court code with name, ie "001C" with "Accomack"
    data["Court"] = court_code[court]
    data = case_matcher(data)
    separate_court_data.append(data) #add to list for concatenation
    
final_cases = pd.concat(separate_court_data, ignore_index = True)

<ipython-input-9-5d20f2d4f79b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judges_court['Year']=judges_court['Year'].astype(int)
<ipython-input-9-5d20f2d4f79b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judges_court['Judge']=judges_court['Judge'].astype(str)
/Users/hinaljajal/Downloads/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_no

In [11]:
final_cases.head()

,Case Number,Court,Last Hearing Date,Charge,Charge Code,Charge Class,Charge Code Section,Concluded By,Sentence Y,Sentence M,...,Probation Type,Probation Y,Probation M,Probation D,Race,Gender,Judge,Year,Judge Full Name,Court Name
0,1700020702,Accomack,05/21/2020,VIOL PROBATION OF MISD OFF,M,UNCLASSIFIED,19.2-306,TRIAL - JUDGE WITH WITNESS,0.0,0.0,...,SUPERVISED,2.0,0.0,0.0,BLACK,MALE,WRL,2020,William Revell Lewis,Accomack
1,0900010001,Accomack,02/25/2010,DISTRIBUTION OF COCAINE,F,NaN,18.2-248,TRIAL - JUDGE WITH WITNESS,5.0,0.0,...,INDEFINITE SUPERVISION,2.0,0.0,0.0,BLACK,MALE,GAT,2010,Glen A Tyler,Accomack
2,9200008800,Accomack,12/07/1992,DIST/CONTROLLED SUBS,F,NaN,18.2-248,GUILTY PLEA,5.0,0.0,...,No Probation,0.0,0.0,0.0,BLACK,MALE,Unknown,1992,Unknown,NaN
3,0600023200,Accomack,10/18/2007,PWID MARIJUANA,M,NaN,18.2-248.1A1,TRIAL - JUDGE WITH WITNESS,0.0,0.0,...,No Probation,0.0,0.0,0.0,WHITE,MALE,GAT,2007,Glen Allen Tyler,Accomack
4,1600018202,Accomack,03/12/2020,DESTRUCT PROP W/INTENT >=$1000,F,CLASS 6,18.2-137,GUILTY PLEA,3.0,0.0,...,SUPERVISED,5.0,0.0,0.0,BLACK,MALE,WRL,2020,William Revell Lewis,Accomack
